<img src="../idp.jpg">

# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Aula 4 - Predição de Desistência de Funcionários</p>

# Introdução

Este é um conjunto de dados criado por cientistas de dados da IBM para analisar os fatores que levam ao desgaste dos funcionários. Neste notebook, construiremos um sistema que prevê a desistência e saída dos funcionários da empresa. Primeiro, faremos um pré-processamento, análise exploratória de dados e, em seguida, construiremos e compararemos 3 modelos de classificação diferentes.

## Dados

Os dados nos trazem várias informações relevantes que levam ao desgaste dos funcionários. Como exemplo: "distância trabalho-casa por função e possível desgaste do funciário" ou "comparação da renda mensal média por educação e desgaste". Este é um conjunto de dados fictício criado por cientistas de dados da IBM.

Link do dataset: https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset

Variáveis de interesse:

**Education**
1 'Below College'
2 'College'
3 'Bachelor'
4 'Master'
5 'Doctor'

**EnvironmentSatisfaction**
1 'Low'
2 'Medium'
3 'High'
4 'Very High'

**JobInvolvement**
1 'Low'
2 'Medium'
3 'High'
4 'Very High'

**JobSatisfaction**
1 'Low'
2 'Medium'
3 'High'
4 'Very High'

**PerformanceRating**
1 'Low'
2 'Good'
3 'Excellent'
4 'Outstanding'

**RelationshipSatisfaction**
1 'Low'
2 'Medium'
3 'High'
4 'Very High'

**WorkLifeBalance**
1 'Bad'
2 'Good'
3 'Better'
4 'Best'

### Roteiro

- Importar bibliotecas
- Análise exploratória dos dados
- Limpeza e tratamento
- Engenharia de atributos 
- Pré-processamento
- Label encoding 
- Data standardization 
- Modelagem
- Treinamento 
- Verificar performance
- Visualização com SHAP

# Importando bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score

In [ ]:
funcionarios = pd.read_csv("ibm-employees.csv")
funcionarios.head()

# Limpeza

In [ ]:
# Vamos verificar se exixtem valores nulos
funcionarios.isnull().sum()

Não há valores nulos no dataset. Que maravilha.

# Análise exploratória dos dados

In [ ]:
# vamos agora fazer nosso famoso EDA automatizado :)
from dataprep.eda import plot, create_report

# Utilizando o método plot para construção do relatório EDA de forma automática dos nomes dos restaurantes
plot(funcionarios)

In [ ]:
create_report(funcionarios)

In [ ]:
# Analisando as variáveis, podemos ver que a target está como categórica. Vamos tranformá-la em numérica.
funcionarios['Attrition'] = funcionarios['Attrition'].factorize(['No','Yes'])[0]
funcionarios.head()

In [ ]:
# Vendo a dimensão do nosso dataset
funcionarios.shape

In [ ]:
# Vamos ver a distruição da target
plt.figure(figsize=(8,8))
pie = funcionarios.groupby('Attrition')['Attrition'].count()
plt.pie(pie, explode=[0.1, 0.1], labels=['No', 'Yes'], autopct='%1.1f%%');

**84% dos funcionários não pediram demissão. 16% pediu demissão.**

### Análise das variáveis x target

In [ ]:
# Vamos agora iniciar a análise das variáveis com a target para entender um pouco melhor os dados
sns.histplot(funcionarios["Age"])

In [ ]:
funcionarios[['Age']].value_counts().sort_values(ascending=False).head(10)

In [ ]:
funcionarios['Age'].describe()

**Maioria dos funcionários tem a idade entre 29 a 40 anos, com média de 37 e mediana de 36 anos**

In [ ]:
# Verificando a hora padrão de trabalho
funcionarios['StandardHours'].value_counts()

In [ ]:
# Vamos ver novamente o nosso mapa de correlação das variáveis
corr = funcionarios.corr()
plt.figure(figsize=(12,12))
sns.heatmap(corr,cbar=True,square=True,fmt='.1f',annot=True,cmap='CMRmap')

**Como podemos ver, não há uma correlação muito forte da target ('Attrition') com nenhuma das colunas numéricas. Mas podemos ver outras correlações como;**
* Funcionários mais seniores têm um total de anos de trabalho mais alto (muito óbvio)
* Classificações de desempenho mais altas levam ao aumento da porcentagem de aumento salarial
* Quanto mais anos um funcionário trabalha, mais sua renda mensal aumenta
* Muitos funcionários permanecem em sua função atual e também sob o mesmo gerente com o passar dos anos, o que significa que eles não recebem promoção e isso pode ser um fator importante que contribui para o pedido de demissão.

**A partir daqui, podemos deduzir que a falta de promoções pode ser um fator crucial para os pedidos de demissão.**

In [ ]:
# Avaliando o tempo na companhia
sns.boxplot(funcionarios["YearsAtCompany"])

**A maioria dos funcionários permanece na empresa por 3-9 anos, sendo a mediana de 5 anos. É considerável também o número de outliers que estão na companhia a mais de 20 anos.**

In [ ]:
# Vamos verificar agora se o funcionário viaja pela companhia x target
sns.countplot(x='BusinessTravel', hue='Attrition', data=funcionarios);

**A maioria dos funcionários que viajam raramente deixam a empresa. A partir do gráfico, podemos dizer também que enviar funcionários em viagens de negócios ou não não faz muita diferença e não tem um efeito significativo nos pedidos de demissão.**

In [ ]:
# Vamos verificar agora o departamento de trabalho x target
plt.figure(figsize=(8,6))
sns.countplot(x='Department', hue='Attrition', data=funcionarios);

In [ ]:
funcionarios['Department'].value_counts()

**A maioria das desistências são do departamento de pesquisa e desenvolvimento. Em segundo lugar temos os funcionários da equipe de vendas. A equipe de Recursos Humanos tem o menor número de desistências. Mas precisamos ter em mente que P&D tem muito mais funcionários do que vendas e RH.**

**Se considerássemos a porcentagem de demissões por departamento, veríamos que o departamento de RH tem a maioria das demissões.**

In [ ]:
# Verificar a target x gênero
funcionarios['Gender'].value_counts()

In [ ]:
sns.countplot(x='Gender', hue='Attrition', data=funcionarios);

**Claramente há mais homens na organização do que mulheres, então as desistências são maiores. Parece que o gênero não seja um fator muito significativo por trás dos pedidos de demissão.**

In [ ]:
# Análise da target x Perfil profisional
plt.figure(figsize=(8,6))
sns.countplot(x='JobRole', hue='Attrition', data=funcionarios);
plt.xticks(rotation=90)

**Entre as funções de trabalho, o destaque fica por conta dos técnicos de laboratório, que pediram demissão. Depois vem o profissional de cientistas de pesquisa, executivos de vendas e representantes de vendas. Poderíamos analisar os salários de cada função e ver se esse pode ser o motivo.**

In [ ]:
# Análise do perfil profissional x salário x target
plt.figure(figsize=(10,6))
sns.barplot(x='JobRole', y='MonthlyIncome', hue='Attrition', data=funcionarios)
plt.xticks(rotation=90)


**Como suspeitávemos, técnicos de laboratório, cientistas de pesquisa e representantes de vendas e executivos têm salários muito baixos e isso pode ser um fator importante por trás dos pedidos de demissão.**

**Além disso, como vimos anteriormente, o departamento de RH teve o maior número de demissões e podemos ver que eles também têm salários muito baixos, então, mais uma vez, isso é algo para se pensar (Sonineide, think about it).**

**Entretanto, a evasão é alta em todos os perfis, inclusive nos salários mais altos, também é um pouco alta.**

In [ ]:
# Analisar o campo de atuação x target
sns.countplot(x='EducationField', hue='Attrition', data=funcionarios);
plt.xticks(rotation=45)

In [ ]:
# Analisando a satisfação do funcionário com o ambiente da empresa. Quase um NPS :)
sns.countplot(x='EnvironmentSatisfaction', data=funcionarios);

In [ ]:
funcionarios['EnvironmentSatisfaction'].value_counts(normalize=True)

**60% dos funcionários estão satisfeitos com a empresa.**

In [ ]:
# Exclusão de algumas variáveis
funcionarios.drop(['BusinessTravel','EducationField','OverTime'],axis=1, inplace=True)

# Modelagem

In [ ]:
# Label encoding das variáveis categóricas

funcionarios['Department'] = preprocessing.LabelEncoder().fit_transform(funcionarios['Department'])
funcionarios['Education'] = preprocessing.LabelEncoder().fit_transform(funcionarios['Education'])
funcionarios['JobRole'] = preprocessing.LabelEncoder().fit_transform(funcionarios['JobRole'])
funcionarios['Gender'] = preprocessing.LabelEncoder().fit_transform(funcionarios['Gender'])
funcionarios['MaritalStatus'] = preprocessing.LabelEncoder().fit_transform(funcionarios['MaritalStatus'])
funcionarios['Over18'] = preprocessing.LabelEncoder().fit_transform(funcionarios['Over18'])

In [ ]:
# Separando as variáveis da target
X = funcionarios.drop(['Attrition'], axis=1) 
y = funcionarios['Attrition'] # Target

In [ ]:
# Dividindo o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 2)

In [ ]:
print("Base de treino", X_train.shape)
print("Base de teste", X_test.shape)

**Vamos rodar 3 diferentes modelos - Random Forest, Logistic Regression e XGBoost para o problema de classificação**

In [ ]:
models = [] # modelos
models.append(("Random Forest", RandomForestClassifier()))
models.append(("Logistic Regression", LogisticRegression()))
models.append(('KNeighbors Classifier',  KNeighborsClassifier()))
models.append(("XGBoost Classifier", XGBClassifier()))


n_folds = 5
results = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=n_folds)
    print("Executando o modelo:", name)
    
    # Validação cruzada
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring="f1_weighted", verbose=0, n_jobs=-1)
    
    # Fit no modelo
    model.fit(X_train,y_train)
    
    # Predição do modelo
    train_pred = model.predict(X_train)
    Training_score = accuracy_score(train_pred,y_train)
    test_pred = model.predict(X_test)
    Test_score = accuracy_score(test_pred,y_test)
    
    results.append(cv_results)
    
    msg = f"Cross_Val Mean: {cv_results.mean()}, Acurácia Treino: {Training_score}, Acurácia Teste: {Test_score}"
    print(msg + "\n")

Opa. Vamos entender um pouco esses resultados. Os 3 classificadores performam praticamente igual. Vamos plotar a matriz de confusão do XGBoost como exemplo.

### Matriz de confusão

<img src="matriz.png">

Verdadeiro Positivo (TP): previsão correta como desistência  
Verdadeiro Negativo (TN): previsão correta como não desistência  
Falso Positivo (FP): Predição incorreta como desistência  
Falso Negativo (FN): Predição Incorreta como não-desistência  

In [ ]:
from sklearn.metrics import confusion_matrix

# Gerando a matriz de confusão
cf_matrix = confusion_matrix(y_test, test_pred)
print(cf_matrix)

In [ ]:
import seaborn as sns
ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')
ax.set_title('Matriz de correlação do XGBoost\n\n');
ax.set_xlabel('\nPredito')
ax.set_ylabel('Valores reais ');
ax.xaxis.set_ticklabels(['Falso','Verdadeiro'])
ax.yaxis.set_ticklabels(['Falso','Verdadeiro'])

plt.show()

Podemos ver claramente que o modelo acerta muita bem os não desistentes, mas os desistentes ele erra de forma considerável. Vamos analisar as outras métricas através do classification_report do sklearn.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, test_pred))

Na primeira linha e segunda coluna temos a Especificidade igual a 96%. Na segunda linha e primeira coluna temos o Precision igual a 48% e na segunda linha e segunda coluna temos o Recall igual a 21%.

**Precision**:  métrica que traz a informação de quantas observações o modelo classificou corretamente como 1.
Observe como o modelo consegue prever melhor a classe 0 do que a classe 1.

**Recall** analisa entre todos os desistentes, quantos realmente o modelo conseguiu prever como desistente.

**F1-Score** é a média harmônica entre o Recall e Precision, ou seja, ela resume as informações dessas duas métricas.

## Utilizando o SHAP para ver quais variáveis tem mais impacto no modelo

https://shap.readthedocs.io/en/latest/index.html

In [ ]:
import shap

explainer = shap.Explainer(model)
shap_values = explainer(X)
shap.summary_plot(shap_values, X)

# Conclusão

O modelo pode ser melhorado através de técnicas de feature engineering ou com hiperparametrização.